# Pipeline推理

`pipeline()` 让使用 `🤗 Hugging Face Hub` 上的任何模型进行任何语言、计算机视觉、语音以及多模态任务的推理都变得非常简单。

即使您对特定的模态没有经验，或者不熟悉模型的源码，您仍然可以使用pipeline()进行推理！

本教程将教您：

1. 如何使用 pipeline() 进行推理。
2. 如何使用特定的 tokenizer(分词器) 或 model(模型)。
3. 如何使用 pipeline() 进行音频、视觉和多模态任务的推理。

```
请查看[pipeline()](https://huggingface.co/docs/transformers/v4.44.2/zh/main_classes/pipelines#transformers.pipeline)文档以获取已支持的任务和可用参数的完整列表。
```

## Pipeline使用

虽然每种任务都有一个关联的 pipeline，但我们可以使用通用的 `pipeline()` 方法，其中包含所有特定任务的 pipelines，**能够根据任务类型自动加载一个默认模型和一个能够进行任务推理的预处理类**。

让我们以使用 pipeline() 进行自动语音识别（ASR）或语音转文本为例。

1. 首先，创建一个 pipeline() 并指定推理任务：

In [ ]:
from transformers import pipeline

transcriber = pipeline(task="automatic-speech-recognition") # 任务：自动语音识别（ASR）

2. 将您的输入传递给 pipeline()。对于语音识别，通常是输入一个音频文件：

In [ ]:
transcriber("https://huggingface.co/datasets/Narsil/asr_dummy/resolve/main/mlk.flac")